# Step 1

**PRD Development**

Generate recruiters and stories..
generate PRD from user stories?
Who does the recruiters need to hire? Are they looking for ____?

In [ ]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact

# Initialize the LLM client. You can change the model here.
client, model_name, api_provider = setup_llm_client(model_name="gemini-2.5-flash")

# Load the artifact from Lab 1
user_stories_data = load_artifact("day1_user_stories.json")
if not user_stories_data:
    user_stories_data = []

**Problem Statement**

In [ ]:
problem_statement = "Assess applicants for recruitment towards specific jobs"

In [ ]:
# TODO: Create a string variable named 'features_prompt'.
# This prompt should ask the LLM to brainstorm features based on the problem_statement.
features_prompt = """ REPLACE """

print("--- Brainstorming Features ---")
brainstormed_features = get_completion(features_prompt, client, model_name, api_provider)
print(brainstormed_features)

# TODO: Create a string variable named 'personas_prompt'.
# This prompt should ask the LLM to identify three user personas based on the problem_statement.
personas_prompt = """ REPLACE"""

print("\n--- Identifying User Personas ---")
user_personas = get_completion(personas_prompt, client, model_name, api_provider)
print(user_personas)

**User Stories**

In [ ]:
# TODO: Create a detailed prompt string named 'json_user_stories_prompt'.
# CREATE BOTH RECRUITERS/HR AND APPLICANTS USER STORIES?
json_user_stories_prompt = f""" REPLACE """

print("--- Generating User Stories as JSON ---")
json_output_str = get_completion(json_user_stories_prompt, client, model_name, api_provider, temperature=0.2)

# Let's try to parse the JSON to see if the LLM followed instructions
try:
    # The LLM might wrap the JSON in markdown fences (```json ... ```).
    # We'll clean that up before parsing.
    if '```' in json_output_str:
        json_output_str = json_output_str.split('```')[1].lstrip('json').strip()
    
    user_stories_json = json.loads(json_output_str)
    print("Successfully parsed LLM output as JSON.")
    
    if user_stories_json:
        print("\n--- Sample User Story ---")
        print(json.dumps(user_stories_json[0], indent=2))
    else:
        print("JSON array is empty.")

except (json.JSONDecodeError, TypeError, IndexError) as e:
    print(f"Error: Failed to parse LLM output as JSON. Error: {e}")
    print("LLM Output was:\n", json_output_str)
    user_stories_json = []

**Validate Stories**

In [ ]:
def validate_and_save_stories(stories_data):
    """Validates the structure of the user stories data and saves it if valid."""
    if not isinstance(stories_data, list) or not stories_data:
        print("Validation Failed: Data is not a non-empty list.")
        return

    required_keys = ['id', 'persona', 'user_story', 'acceptance_criteria']
    all_stories_valid = True

    # TODO: Implement the validation logic inside this function.
    # 1. Loop through each story in the 'stories_data' list.
    # 2. For each story, check if it contains all the 'required_keys'.
    # 3. Also check if the 'acceptance_criteria' list is not empty.
    # 4. If a story is invalid, print an error message and set 'all_stories_valid' to False.
    #    (You can use 'continue' to skip to the next story).

    # Your validation code here
    for i, story in enumerate(stories_data, 1):
        # Check if story is a dictionary
        if not isinstance(story, dict):
            print(f"Error: Story {i} is not a dictionary.")
            all_stories_valid = False
            continue
            
        # Check if all required keys are present
        missing_keys = [key for key in required_keys if key not in story]
        if missing_keys:
            print(f"Error: Story {i} is missing required keys: {missing_keys}")
            all_stories_valid = False
            continue
            
        # Check if acceptance_criteria is not empty
        acceptance_criteria = story.get('acceptance_criteria')
        if not acceptance_criteria or (isinstance(acceptance_criteria, list) and len(acceptance_criteria) == 0):
            print(f"Error: Story {i} has empty acceptance_criteria.")
            all_stories_valid = False
            continue
            
        print(f"Story {i}: Valid")

    if all_stories_valid:
        print("\nAll user stories passed validation.")
        artifact_path = "artifacts/day1_user_stories.json"
        
        # TODO: Call the save_artifact function from utils.py to save the data.
        # Remember to convert the Python list back to a JSON string using json.dumps().
        save_artifact(json.dumps(stories_data, indent=2), artifact_path)
        print(f"User stories saved to {artifact_path}")
        
    else:
        print("\nValidation failed. Artifact not saved.")

# Run the validation on the JSON data from the previous step
if 'user_stories_json' in locals() and user_stories_json:
    validate_and_save_stories(user_stories_json)
else:
    print("Skipping validation as user_stories_json is empty or not defined.")

**Generate PRD**


In [ ]:
# TODO: Write a prompt to generate a simple PRD.
simple_prd_prompt = f""" REPLACE"""

print("--- Generating Simple PRD ---")
if user_stories_data:
    simple_prd_output = get_completion(simple_prd_prompt, client, model_name, api_provider)
    print(simple_prd_output)
else:
    print("Skipping PRD generation because user stories are missing.")

In [ ]:
# Load the PRD template directly using the project root path
template_path = os.path.join(project_root, "templates", "prd_template.md")
try:
    with open(template_path, 'r', encoding='utf-8') as f:
        prd_template_content = f.read()
    print(f"Successfully loaded template from: {template_path}")
except FileNotFoundError:
    print(f"Template file not found at: {template_path}")
    prd_template_content = ""
except Exception as e:
    print(f"Error loading template: {e}")
    prd_template_content = ""

# TODO: Write a prompt to populate the PRD template.
template_prd_prompt = f""" REPLACE """

print("--- Generating PRD from Template ---")
if user_stories_data and prd_template_content:
    prd_from_template_output = get_completion(template_prd_prompt, client, model_name, api_provider)
    print(prd_from_template_output)
else:
    print("Skipping PRD generation because user stories or template are missing.")
    prd_from_template_output = ""

In [ ]:
# TODO: Write a prompt to generate a Pydantic model for the PRD.
# Tip: Be specific. Tell the LLM to create a class named 'ProductRequirementsDocument' and to use appropriate types from Python's 'typing' library.
pydantic_model_prompt = f""" REPLACE"""

print("--- Generating Pydantic Model for PRD ---")
if prd_template_content:
    pydantic_model_code = get_completion(pydantic_model_prompt, client, model_name, api_provider)
    
    # Clean up the code if it's wrapped in markdown fences
    if '```' in pydantic_model_code:
        pydantic_model_code = pydantic_model_code.split('```')[1].lstrip('python').strip()
    
    print("\n--- Generated Pydantic Model ---")
    print(pydantic_model_code)

    # Save the generated Pydantic model code to a file.
    model_path = "app/validation_models/prd_model.py"
    save_artifact(pydantic_model_code, model_path)
else:
    print("Skipping Pydantic model generation because template is missing.")

# Finally, save the completed PRD from the intermediate challenge
if prd_from_template_output:
    save_artifact(prd_from_template_output, "day1_prd.md")

## Step 2

**Generate design and architecture


In [ ]:
# TODO: Write a prompt to generate a markdown ADR template.
adr_template_prompt = """REPLACE """

print("--- Generating ADR Template ---")
adr_template_content = get_completion(adr_template_prompt, client, model_name, api_provider)
print(adr_template_content)

# Save the artifact (overwrite if it already exists to avoid ArtifactError)
if adr_template_content:
    save_artifact(adr_template_content, "templates/adr_template.md", overwrite=True)

In [ ]:
# TODO: Write a prompt to research database options.
db_research_prompt = """
REPLACE"""

print("--- Researching Database Options ---")
db_research_output = get_completion(db_research_prompt, client, model_name, api_provider)
print(db_research_output)

In [ ]:
from datetime import date

adr_template = load_artifact("artifacts/adr_template.md")

# TODO: Write a prompt to synthesize the final ADR.
today = date.today().isoformat()

synthesis_prompt = f"""REPLACE"""

print("--- Synthesizing Final ADR ---")
if adr_template and 'db_research_output' in locals() and db_research_output:
    final_adr = get_completion(synthesis_prompt, client, model_name, api_provider)
    print(final_adr)
    save_artifact(final_adr, "artifacts/adr_001_database_choice.md", overwrite=True)
else:
    print("Skipping ADR synthesis because template or research is missing.")